In [129]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [130]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')
lat_long = pd.read_csv('http://cocl.us/Geospatial_data')

In [135]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [136]:
toronto_neigh = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df = pd.DataFrame.from_dict(dict(toronto_neigh))

In [137]:
toronto_df = toronto_df[(toronto_df.Borough != 'Not assigned')]
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [138]:
toronto_df.insert(3, "Latitude", lat_long["Latitude"], allow_duplicates = False)

In [139]:
toronto_df.insert(4, "Longitude", lat_long["Longitude"], allow_duplicates = False)

In [140]:
toronto_df.groupby(['PostalCode','Borough', 'Neighborhood', 'Latitude'], as_index=False).agg(lambda x: ','.join(x))
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.744734,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029


In [141]:
for index, row in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.744734,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.716316,-79.239476
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
11,M3B,North York,Don Mills,43.750072,-79.295849
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.794200,-79.262029
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.781638,-79.304302


In [142]:
toronto_df.shape

(103, 5)